In [1]:
import os
import pickle
from pathlib import Path

In [2]:
from imports import *
from data_loader import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/eitan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
import optuna
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

import onnxruntime as rt 
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType

from sklearn.preprocessing import StandardScaler


SEED = 42

In [4]:
trainX, testX, trainY, testY = sql_vectorized(db_file=file_name, query_limit=100000)

# Standardize the input features
scaler = StandardScaler(with_mean = False)
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

file_name = 'star_reviews.db'
X_raw, Y_raw = sql_query_raw(db_file=file_name, query_limit=100000)

# Split the data into training and testing sets
raw_trainX, raw_testX, raw_trainY, raw_testY = train_test_split(X_raw, Y_raw, test_size=0.2, random_state=42)

# Step 1: Convert raw text data to TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
X_tfidf_train = tfidf_vectorizer.fit_transform(raw_trainX)  # Fit on training data
X_tfidf_test = tfidf_vectorizer.transform(raw_testX)       # Transform test data using the same vectorizer

# Step 2: Train logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_tfidf_train, raw_trainY)  # Train on training data and labels

# Step 3: Make predictions on the test set
pred_labels = log_reg.predict(X_tfidf_test)

# Step 4: Evaluate the model using F1 scores
f1_micro = f1_score(raw_testY, pred_labels, average="micro") 
f1_macro = f1_score(raw_testY, pred_labels, average="macro")

print(f"F1 Micro: {f1_micro}")
print(f"F1 Macro: {f1_macro}")


F1 Micro: 0.6476
F1 Macro: 0.5608296542359805


In [7]:
log_reg = LogisticRegression(
    random_state=SEED, solver="saga", max_iter=5000, multi_class='ovr'
)

log_reg.fit(trainX, trainY)

# Predict on the test set
pred_labels = log_reg.predict(testX)

# F1 scores for multiclass classification
f1_micro = f1_score(testY, pred_labels, average="micro")
f1_macro = f1_score(testY, pred_labels, average="macro")

print(f"F1 Micro: {f1_micro}")
print(f"F1 Macro: {f1_macro}")

# F1 Micro: 0.702125
# F1 Macro: 0.6498956429594761


F1 Micro: 0.63455
F1 Macro: 0.5492665574986099


In [8]:


log_reg = LogisticRegression(
    random_state=SEED, solver="saga", max_iter=10000, n_jobs=-1
)
log_reg.fit(trainX, trainY)

pred_labels = log_reg.predict(testX)
f1_micro = f1_score(testY, pred_labels, average="micro")
f1_macro = f1_score(testY, pred_labels, average="macro")

print(f"F1 Micro: {f1_micro}")
print(f"F1 Macro: {f1_macro}")

# F1 Micro: 0.708125
# F1 Macro: 0.6640736324269323

F1 Micro: 0.64075
F1 Macro: 0.56679123342431
